# <span style='color:SteelBlue'>P6 - Classifiez automatiquement des biens de consommation (part 3)</span>

<img src="https://user.oc-static.com/upload/2019/02/24/15510259240381_Projet%20textimage%20logo.png" width="600" height="400">

# <span style='background:red; color:black'>a MAJ</span>

# <span style='background:white; color:black'>Sommaire</span>

Importations des librairies utilisees dans ce notebook <br>

Paramètres d'affichage du notebook <br>

Fonctions utilisees dans ce notebook <br>

Importation des données <br>

**Etape 1 : Description des donnees**
- _1/ etc...


Conclusion

# <span style='background:blue'>Introduction</span>

L'entreprise **"place de marché"** est une entreprise qui souhaite lancer une
marketplace e-commerce. Sur cette place de marché anglophone, des vendeurs
proposent des articles à des acheteurs en postant une photo et une description.
Pour rendre l’expérience utilisateur des vendeurs (faciliter la mise en ligne
de nouveaux articles) et des acheteurs (faciliter la recherche de produits) 
la plus fluide possible, et dans l'optique d'un passage à l'échelle, 
il devient nécessaire **d'automatiser l'attribution de la catégorie** d'un 
article (actuellement effectuée manuellement par les vendeurs, donc peu 
fiable).
___
**Missions**<br>
1/ Faire une **étude de la faisabilité d’un moteur de classification**
d’article basé sur une **description** pour automatiser l’attribution
de catégorie de l’article. 

2/ Faire une **étude de la faisabilité d’un moteur de classification**
d’article basé sur une **image** pour automatiser l’attribution
de catégorie de l’article. 

3/ **Réaliser une classification supervisée** à partir des images par la mise
en place d’une data augmentation afin d’optimiser le modèle. 

4/ **Extraire les 10 premiers produits à base de « champagne »** issus de
l’API fournie dans un fichier “.csv”, contenant pour chaque produit les
données suivantes : foodId, label, category, foodContentsLabel, image.
___
Ce notebook presente la 3ème partie concernant la réalisation de la
classification...

# <span style='background:red'>a terminer</span>

___
**Source des datasets**<br>
Source : Non précisé<br>
___

<div class="alert alert-block alert-warning">
<b>Formattage du notebook au PEP8 => lancer cette ligne de code dans le
terminal VSC : </b><br>
nbqa black Berthe_Pierrick_1_notebook_exploration_112023.ipynb</div>

# <span style='background:grey'>Importations des librairies utilisees dans ce notebook</span>

In [ ]:
# Librairies generales
import os.path
import sys
from IPython.display import display
import time
from datetime import datetime
from tqdm import tqdm
import concurrent.futures

# Librairies data science
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# CNN Transfer learning
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Affichage plot dans le notebook
%matplotlib inline

# Versions
print("Version des librairies utilisees :")
print("Python        : " + sys.version)
print("Matplotlib    : " + mpl.__version__)
print("NumPy         : " + np.version.full_version)
print("Pandas        : " + pd.__version__)
print("Seaborn       : " + sns.__version__)
print("Tensorflow    : " + tf.__version__)

# Nombre de coeurs
nbr_coeur = os.cpu_count()
print(f"\nNombre de coeur disponible(s) sur ce PC: {nbr_coeur} coeur(s)")

# Afficher heure lancement
maintenant = datetime.now().isoformat()
print("\nCode lance le : " + maintenant)

# Enregistrer l'heure de debut
start_time = time.time()

# Activer la verification PEP8
print("")
%load_ext pycodestyle_magic
%pycodestyle_on

<div class="alert alert-block alert-warning">
<b>Formattage du notebook au PEP8 => lancer cette ligne de code dans le 
terminal VSC : </b> <br>
nbqa black nom_fichier.ipynb</div>

# <span style='background:grey'>Paramètres d'affichage</span>

# <span style='background:grey'>Fonctions</span>

In [1]:
def calculer_duree_notebook(start_time):
    """
    Cette procédure calcule et affiche la durée totale depuis un temps de
    début donné.

    Args:
        start_time (float): Le temps de début en secondes depuis l'époque.

    Returns:
        None
    """

    # Calculer la durée totale
    total_time = time.time() - start_time

    # Convertir en minutes et secondes
    minutes, seconds = divmod(total_time, 60)

    # Afficher la durée totale
    message = (
        "Durée totale pour faire tourner le notebook : "
        "{0} minute(s) {1} seconde(s)"
    )
    print(message.format(int(minutes), int(seconds)))

# <span style='background:grey'>Importation des donnees</span>

Importation des fichiers CSV avec Pandas

In [ ]:
# Definir le chemin
chemin = "./data/source/"

# Obtenir une liste de tous les fichiers dans le repertoire
fichiers = os.listdir(chemin)

# Initialiser un dictionnaire vide et display de la liste des fichiers importe
data_brut = {}
display("===> Liste des fichiers importes: <====\n")

# POUR chaque fichier
for fichier in fichiers:

    # Traiter uniquement les fichiers .csv
    if fichier.endswith(".csv"):
        display(fichier)

        # Lire le fichier csv et le stocker dans le dictionnaire
        data_brut[fichier] = pd.read_csv(
            chemin + fichier,
            sep=",",
            encoding="utf-8"
        )

# <span style='background:blue'>Etape 1 : Présentation du dataset</span>

## <span style='background:green'>1/ Aperçu tableau</span>

Compétence OCR:

**Définir la stratégie d’élaboration d’un modèle d'apprentissage profond, concevoir ou ré-utiliser des
modèles pré-entraînés (transfer learning) et entraîner des modèles afin de réaliser une analyse
prédictive.**
- CE1 Le candidat a défini sa stratégie d’élaboration d’un modèle pour répondre à un besoin métier (par
exemple : choix de conception d’un modèle ou ré-utilisation de modèles pré-entraînés).
- CE2 Le candidat a identifié la ou les cibles.
- CE3 Le candidat a réalisé la séparation du jeu de données en jeu d’entraînement, jeu de validation et
jeu de test.
- CE4 Le candidat s'est assuré qu'il n’y a pas de fuite d’information entre les jeux de données
(entraînement, validation et test).
- CE5 Le candidat a testé plusieurs modèles d’apprentissage profond (par exemple à l’aide de la librairie
Tensorflow / Keras) en partant du plus simple vers les plus complexes.
- CE6 Le candidat a mis en oeuvre des modèles à partir de modèles pré-entraînés (technique de
Transfer Learning)

**Évaluer la performance des modèles d’apprentissage profond selon différents critères (scores,
temps d'entraînement, etc.) afin de choisir le modèle le plus performant pour la problématique
métier.**
- CE1 Le candidat a choisi une métrique adaptée à la problématique métier, et sert à évaluer la
performance des modèles
- CE2 Le candidat a explicité le choix de la métrique d’évaluation
- CE3 Le candidat a évalué la performance d’un modèle de référence et sert de comparaison pour
évaluer la performance des modèles plus complexes
- CE4 Le candidat a calculé, hormis la métrique choisie, au moins un autre indicateur pour comparer les
modèles (par exemple : le temps nécessaire pour l’entraînement du modèle)
- CE5 Le candidat a optimisé au moins un des hyperparamètres du modèle choisi (par exemple : le choix
de la fonction Loss, le Batch Size, le nombre d'Epochs)

**Utiliser des techniques d’augmentation des données afin d'améliorer la performance des modèles.**
- CE1 Le candidat a utilisé plusieurs techniques d’augmentation des données (ex. pour des images :
rotation, changement d’échelle, ajout de bruit…).
- CE2 Le candidat a présenté une synthèse comparative des améliorations de performance grâce aux
différentes techniques d'augmentation de données utilisées (maîtrise de l’overfitting, meilleur score).

In [ ]:
# Afficher temps d'exécution du notebook
calculer_duree_notebook(start_time)